In [1]:
import os
import numpy as np
from dataclasses import dataclass, fields

from VASP_job.code.main import VASP_job

In [2]:
executable_path= "/home/emendive/workbench/work/from_claix/codes/VASP/vasp.5.4.4-flag4/bin"
potential_path = "/home/emendive/workbench/work/from_claix/codes/VASP/potentials/potpaw_PBE"
vasp = VASP_job(verbose="low", executable_path=executable_path, potential_path=potential_path)

In [4]:
vasp.cwd

'/home/emendive/workbench/devel/gitclones/VASP_job/'

In [3]:
vasp.write_inputs_and_job()

In [15]:
fields(vasp.job_parameters)[6].name.replace("_", "-")

'mem-per-cpu'

In [3]:
vasp.cwd = "/u/edmen"
print(vasp.cwd)
print(vasp.KPOINTS_file)

/u/edmen/
/u/edmen/KPOINTS


In [4]:
vasp.standard_INCAR_parameters

standard_INCAR_parameters(SYSTEM='my material #jobname', PREC='Accurate', ALGO='Fast', NPAR='8', LREAL='.FALSE.', LWAVE='.FALSE.', LCHARG='.TRUE.', LORBIT='10', ISMEAR='1', SIGMA='0.03', ISTART='0', ICHARG='2', ISPIN='2', ENCUT='500', EDIFF='1e-6', NELM='200', LMAXMIX='6', LNONCOLLINEAR='.TRUE.')

In [6]:
vasp.structure.kpoints = "1 1 1"
vasp.structure.kpoints

'1 1 1'

In [15]:
vasp.structure.elements = ["Ni"]
vasp.structure.potential_files.Ni = "Ni"

In [16]:
vasp.structure.write_POTCAR()
vasp.structure.write_KPOINTS()

In [11]:
vasp.structure.potential_files.Mn = 'Mn'
vasp.structure.potential_files.Mn

'Mn'

In [12]:
vasp.structure.write_POTCAR()